## Setup enviroment

In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append(r'/home/jeanlr/projetos/lending-club/global')
from util import reduce_mem_usage
import json

/home/jeanlr/projetos/lending-club/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read train parquet file

In [3]:
train_df = pd.read_parquet('../data/raw/lending_club_case_train_dataset.parquet')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1765426 entries, 0 to 2925491
Columns: 142 entries, id to default
dtypes: Int16(54), Int32(15), Int64(15), category(22), datetime64[ns](9), float16(23), float32(4)
memory usage: 1.0 GB


## Read test parquet file

In [4]:
test_df = pd.read_parquet('../data/raw/lending_club_case_case_test_dataset.parquet')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 538826 entries, 472065 to 1082981
Columns: 141 entries, id to debt_settlement_flag
dtypes: Int16(54), Int32(15), Int64(15), category(22), datetime64[ns](9), float16(22), float32(4)
memory usage: 389.5 MB


## Create new features for train dataframe

In [5]:
## Features baseadas em dados demográficos e de empréstimo
# Razão entre a renda anual (annual_inc) e o valor do comprometido (funded_amnt)
train_df['income_to_funded_ratio'] = train_df['annual_inc'] / train_df['funded_amnt']
#Razão entre as dívidas (dti) e a renda anual (annual_inc)
train_df['debt_to_income_ratio'] = train_df['dti'] / train_df['annual_inc']
#Razão entre (funded_amnt) e a renda anual (annual_inc)
train_df['funded_amnt_per_income'] = train_df['funded_amnt'] / train_df['annual_inc']
#Média entre (fico_range_high) e a renda anual (fico_range_low)
train_df['fico_avg'] = (train_df['fico_range_high'] / train_df['fico_range_low'])/2

## Features baseadas em dados de histórico de crédito
# Razão entre o saldo atual (revol_bal) e o limite de crédito (total_rev_hi_lim)
train_df['credit_utilization_ratio'] = train_df['revol_bal'] / train_df['total_rev_hi_lim']
# Soma das contas abertas (open_acc) e contas fechadas (total_acc)
train_df['total_credit_lines'] = train_df['open_acc'] + train_df['total_acc']
# Razão entre o número de vezes que o cliente esteve em atraso (delinq_2yrs) e o número total de contas (total_acc)
train_df['delinquency_ratio'] = train_df['delinq_2yrs'] / train_df['total_acc']

## Features baseadas em dados de características do empréstimo
# Razão entre a taxa de juros (int_rate) e a renda anual (annual_inc)
train_df['int_rate_to_income_ratio'] = train_df['int_rate'] / train_df['annual_inc']

## Features baseadas em dados de comportamento sobre o Pagamento
# Número de registros públicos (pub_rec) ponderado pelo valor do comprometido (funded_amnt)
train_df['public_records_impact'] = train_df['pub_rec'] * train_df['funded_amnt']

## Features baseadas em dados de comportamento sobre o linhas de crédito
#Razão entre (num_actv_bc_tl) e a renda anual (num_bc_tl)
train_df['pct_active_bc'] = train_df['num_actv_bc_tl'] / train_df['num_bc_tl']

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1765426 entries, 0 to 2925491
Columns: 152 entries, id to pct_active_bc
dtypes: Float64(6), Int16(55), Int32(16), Int64(15), category(22), datetime64[ns](9), float16(23), float32(6)
memory usage: 1.1 GB


## Create new features for test dataframe

In [7]:
## Features baseadas em dados demográficos e de empréstimo
# Razão entre a renda anual (annual_inc) e o valor comprometido (funded_amnt)
test_df['income_to_funded_ratio'] = test_df['annual_inc'] / test_df['funded_amnt']
#Razão entre as dívidas (dti) e a renda anual (annual_inc)
test_df['debt_to_income_ratio'] = test_df['dti'] / test_df['annual_inc']
#Razão entre (funded_amnt) e a renda anual (annual_inc)
test_df['funded_amnt_per_income'] = test_df['funded_amnt'] / test_df['annual_inc']
#Média entre (fico_range_high) e a renda anual (fico_range_low)
test_df['fico_avg'] = (test_df['fico_range_high'] / test_df['fico_range_low'])/2

## Features baseadas em dados de histórico de crédito
# Razão entre o saldo atual (revol_bal) e o limite de crédito (total_rev_hi_lim)
test_df['credit_utilization_ratio'] = test_df['revol_bal'] / test_df['total_rev_hi_lim']
# Soma das contas abertas (open_acc) e contas fechadas (total_acc)
test_df['total_credit_lines'] = test_df['open_acc'] + test_df['total_acc']
# Razão entre o número de vezes que o cliente esteve em atraso (delinq_2yrs) e o número total de contas (total_acc)
test_df['delinquency_ratio'] = test_df['delinq_2yrs'] / test_df['total_acc']

## Features baseadas em dados de características do empréstimo
# Razão entre a taxa de juros (int_rate) e a renda anual (annual_inc)
test_df['int_rate_to_income_ratio'] = test_df['int_rate'] / test_df['annual_inc']

## Features baseadas em dados de comportamento sobre o Pagamento
# Número de registros públicos (pub_rec) ponderado pelo valor do comprometido (funded_amnt)
test_df['public_records_impact'] = test_df['pub_rec'] * test_df['funded_amnt']

## Features baseadas em dados de comportamento sobre o linhas de crédito
#Razão entre (num_actv_bc_tl) e a renda anual (num_bc_tl)
test_df['pct_active_bc'] = test_df['num_actv_bc_tl'] / test_df['num_bc_tl']

In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 538826 entries, 472065 to 1082981
Columns: 151 entries, id to pct_active_bc
dtypes: Float64(6), Int16(55), Int32(16), Int64(15), category(22), datetime64[ns](9), float16(22), float32(6)
memory usage: 425.5 MB


## Drop columns with contains future information

In [9]:
train_df_v2 = train_df.copy()



# Lista de colunas com dados coletados após a concessão do crédito, ou seja, após o processo de aplicação (application)
after_grant_cols= ['loan_amnt','out_prncp', 'out_prncp_inv', 'last_fico_range_low',
                   'last_fico_range_high','total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
                   'total_rec_late_fee', 'recoveries', 'collection_recovery_fee','tot_coll_amt','last_pymnt_amnt',
                   'total_il_high_credit_limit', 'hardship_flag', 'installment',
                   'revol_util','collections_12_mths_ex_med',
                   'last_credit_pull_d', 'next_pymnt_d', 'last_pymnt_d', 'collection_recovery_fee'
                   ]
# Removendo as variáveis relacionadas ao plano de dificuldades (covid)
hardship_cols = ['hardship_flag','hardship_type','hardship_reason','hardship_status','deferral_term',
                 'hardship_amount','hardship_start_date','hardship_end_date','payment_plan_start_date',
                 'hardship_length','hardship_dpd','hardship_loan_status','orig_projected_additional_accrued_interest',
                 'hardship_payoff_balance_amount','hardship_last_payment_amount','debt_settlement_flag']

useless_cols = ['id', 'initial_list_status', 'title', 'purpose', 'url', 'pymnt_plan', 'loan_status', 'emp_title']

adress_cols = ['zip_code', 'addr_state']

date_cols = list(train_df_v2.select_dtypes(include='datetime').columns)

# Colunas a serem dropadas
cols_to_drop = after_grant_cols + hardship_cols + useless_cols + date_cols + adress_cols

In [10]:
# Cols that will not be used during the traning of the model
# Saving the features list
features_list = list(train_df_v2.drop(columns=cols_to_drop).columns)
with open('../artifacts/features_list.json', 'w') as f:
    json.dump(features_list, f)

## Saving the processed ABT

In [11]:
# Train
train_df.to_parquet('../data/processed/feature_engineering_train.parquet')

#Test
test_df.to_parquet('../data/processed/feature_engineering_test.parquet')